# Pipeline Simulation - Test

In [11]:
import numpy as np
import torch
from feature_detection import Net
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from _datasets import ObjectPointCloudDataset

## Sanity Check

### Label Assignation

In [12]:
import os
import re

names = os.listdir('ycb')

labels = {}

for name in names:
    id = int(name[:3])
    
    labels[id] = name


### Models Instantiation

In [19]:
pc = np.load("sanity_check_3.npy")
centroid = np.mean(pc, axis=0)
pc = pc - centroid
normalized = pc / np.std(pc)
data = Data(x = torch.tensor(pc, dtype=torch.float)) 
loader = DataLoader(dataset=[data], batch_size=1)

In [20]:
# Load the model from pt files
model_paths = [path for path in os.listdir('models')]
model_paths = ['model.pt', 'modelv2.pt']
models = []
for path in model_paths:
    model = Net()
    model.to('cuda')
    model.load_state_dict(torch.load('models/'+path)['model_state_dict'])
    model.eval()
    models.append(model)

# Loading point clouds from nympy file
# pc = np.load("sanity_check.npy")
# data = Data(x = torch.tensor(pc, dtype=torch.float))
# loader = DataLoader(dataset=[data], batch_size=1)

for i in range(len(models)):
    for d in loader:
        d = d.to('cuda')
        result = models[i](d)[0].max(1)[1]

    print(f"{model_paths[i]} => ", result, " : ", labels[result.item()])

model.pt =>  tensor([7], device='cuda:0')  :  007_tuna_fish_can.stl
modelv2.pt =>  tensor([62], device='cuda:0')  :  062_dice.stl


In [ ]:
model = Net()
model.to('cuda')
model.load_state_dict(torch.load('models/modelv2.pt')['model_state_dict'])
model.eval()
models.append(model)


# Loading test dataset frompt files
test_dataset = ObjectPointCloudDataset(root = '.', 
                                    chunk = (87984, 109980), 
                                    sample_count = 512,
                                    output_name = 'testv2'
                                    )

loader = DataLoader(test_dataset,
                    batch_size=1)

point_clouds = []
label = []

test_num = 0
for d in loader:
    test_num += 1
    d = d.to('cuda')
    result = models[i](d)[0].max(1)[1]

    print(result, " : ", labels[result.item()], " | ", d.y)
    print(d.x.dtype)

    pc = d.x.cpu().numpy()

    point_clouds.append(pc)
    label.append(labels[result.item()])

    if test_num > 9:
        break

In [ ]:
for i in range(len(point_clouds)):
    np.save(f"pcs/{label[i]}_{i}", point_clouds[i])

In [ ]:
files = os.listdir('../dataset/v4/')
labels = set()

for file in files[:8794]:
    labels.add(file[:3])

print(labels)

test = set()
for file in files[8794:]:
    test.add(file[:3])

print(test)


In [ ]:
repetitions = {}

for label in files:
    try:
        repetitions[label[:3]] += 1
    except:
        repetitions[label[:3]] = 1

print(repetitions)

In [ ]:
import open3d as o3d

def visualize_point_cloud(pc):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    o3d.visualization.draw_geometries([pcd])

In [ ]:
# %matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_point_cloud(mat):
    # Create a 3D figure
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # Define the x, y, z coordinates of the point cloud
    x = mat[:, 0]
    y = mat[:, 1]
    z = mat[:, 2]

    # Plot the point cloud data
    ax.scatter(x, y, z, s=1)

    # Set the axis labels
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    ax.scatter(0, 0, 0, s=10)

    ax.set_xlim(-0.3, 0.3)
    ax.set_ylim(-0.3, 0.3)
    ax.set_zlim(-0.3, 0.3)

    # Show the plot
    plt.show()
    # print

plot_point_cloud(pc)
